In [39]:
import numpy as np
from pandas import DataFrame
data_iris = pd.read_csv('iris.csv')
X_iris = data_iris.iloc[:, 1:-1]
s = data_iris.iloc[:, -1]
y_iris=[]
for i in range(0,len(s)):
    if s[i]=='Iris-setosa':
        y_iris.append(0)
    if s[i]=='Iris-versicolor':
        y_iris.append(1)
    if s[i]=='Iris-virginica':
        y_iris.append(2)


y_iris = DataFrame (y_iris)
y_iris.columns = ['target']


In [40]:
# --------------------------------------------------------
### The Following section splits the iris dataset directly from sklean
# --------------------------------------------------------
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.2, random_state=42)

# --------------------------------------------------------
# display train-test split information
# --------------------------------------------------------
print ('Information after train-test split:')
print('The train-set includes %d instances and %d corresponding categories\n' %(X_train.shape[0],y_train.shape[0]))
print('The test-set includes %d instances and %d corresponding categories\n' %(X_test.shape[0],y_test.shape[0]))

# --------------------------------------------------------
## concatinate the X_train and y_train for Naive Bayes training:
# --------------------------------------------------------
train_set = pd.concat((X_train, y_train), axis=1)
# --------------------------------------------------------
if 'datasets' in sys.modules:
    del (datasets)
if 'train_test_split' in sys.modules:
    del (train_test_split)
sys_modules = list(sys.modules.keys())
for mdl in sys_modules:
    if mdl.startswith('sklearn.'):
        del(sys.modules[mdl]) 
del (sklearn)
if 'sklearn' in sys.modules:
    del (sys.modules['sklearn'])
# --------------------------------------------------------
# Display the first few rows of the training-set:
# --------------------------------------------------------
print('First few rows of unified train-set:')
train_set.head()

Information after train-test split:
The train-set includes 120 instances and 120 corresponding categories

The test-set includes 30 instances and 30 corresponding categories

First few rows of unified train-set:


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,target
22,4.6,3.6,1.0,0.2,0
15,5.7,4.4,1.5,0.4,0
65,6.7,3.1,4.4,1.4,1
11,4.8,3.4,1.6,0.2,0
42,4.4,3.2,1.3,0.2,0


In [41]:
def calcCategoryPriors(trainingSet):
    
    yTrain = trainingSet[['target']]
    total=yTrain.shape[0]
    uniqueClasses = np.unique(yTrain['target'].values)
    helpLi=[]
    li=[]
    for m in uniqueClasses:
        helpLi.append(0)
        li.append(0)
    
    for i in yTrain['target']:
        for x in uniqueClasses:
            if i==uniqueClasses[x]:
                helpLi[x]+=1
    for x in range(len(li)):
        li[x]=helpLi[x]/total
    return li

In [42]:

arrPriors = calcCategoryPriors(train_set)
priorClass_0 = arrPriors[0]
print ('testing for expected class prior ...')
assert int(priorClass_0*100)==33,'wrong prior for class-0'
print ("... 'calcCategoryPriors' test passed successfully :-)")
print ('prior for category 0: %f' %(priorClass_0))

testing for expected class prior ...
... 'calcCategoryPriors' test passed successfully :-)
prior for category 0: 0.333333


In [43]:
def calcMeanLikelihood(trainingSet):
    yTrain = trainingSet[['target']]
    meandf=pd.DataFrame(index=np.unique(yTrain['target'].values),columns=trainingSet.columns)
    meandf=meandf.drop(columns=['target'])
    for i in range(len(meandf)):
        meandf0=trainingSet.loc[trainingSet['target'].values == i]
        meandf0=meandf0.drop(columns=['target'])
        for col in meandf0.columns:
            meandf.loc[i,col]=meandf0[col].mean()
    return meandf

In [44]:

meanLiklihoodDf = calcMeanLikelihood(train_set)
likelihood_petalLength_class1 = meanLiklihoodDf.iloc[1,2]
print ('testing for expected mean likelihood estimation ...')
assert int(likelihood_petalLength_class1*10)==42,'wrong mean likelihood estimation for petalLength_class1'
print ("... 'calcMeanLikelihood' test passed successfully :-)")
print ('likelihood for the mean of petal length for category 1 is estimated as: %f' %(likelihood_petalLength_class1))

testing for expected mean likelihood estimation ...
... 'calcMeanLikelihood' test passed successfully :-)
likelihood for the mean of petal length for category 1 is estimated as: 4.241463


In [45]:

def calcStdLikelihood(trainingSet):
    yTrain = trainingSet[['target']]
    std=pd.DataFrame(index=np.unique(yTrain['target'].values),columns=trainingSet.columns)
    std=std.drop(columns=['target'])
    for i in range(len(std)):
        std0=trainingSet.loc[trainingSet['target'].values == i]
        std0=std0.drop(columns=['target'])
        for col in std0.columns:
            std.loc[i,col]=std0[col].std()
    return std

In [46]:
stdLiklihoodDf = calcStdLikelihood(train_set)
likelihood_petalLength_class1 = stdLiklihoodDf.iloc[1,2]
print ('testing for expected std likelihood estimation ...')
assert int(likelihood_petalLength_class1*10)==4,'wrong std likelihood estimation for petalLength_class1'
print ("... 'calcStdLikelihood' test passed successfully :-)")
print ('likelihood for the std of petal length for category 1 is estimated as: %f' %(likelihood_petalLength_class1))

testing for expected std likelihood estimation ...
... 'calcStdLikelihood' test passed successfully :-)
likelihood for the std of petal length for category 1 is estimated as: 0.481132


In [47]:

def fit(trainingSet):
    """
    1. Calculate the class priors of the training set, using the 'calcCategoryPriors' method.
    2. Calculate the mean of the training set per feature per class, using the 'calcMeanLikelihood' method.
    3. Calculate the std of the training set per feature per class, using the 'stdLiklihoodDf' method.
    """
    arrPriors = calcCategoryPriors(trainingSet)
    meanLiklihoodDf = calcMeanLikelihood(trainingSet)
    stdLiklihoodDf = calcStdLikelihood(trainingSet)
    
    return meanLiklihoodDf, stdLiklihoodDf, arrPriors

In [48]:
def calcGaussianProb(xFeatureVal, mean, std):
    exponent = np.exp(-((xFeatureVal-mean)**2 / (2 * std**2 )))
    return (1 / ((2 * np.pi)**(1/2) * std)) * exponent

In [49]:
def calcAposteriorProbs(XTest, arrTrainedClassPriors, dfTrainedMean, dfTrainedStd, categories):
    numClasses = len(categories)
    dfProbPerTestInstPerClass = pd.DataFrame(np.zeros((XTest.shape[0], numClasses)), columns=categories, index=XTest.index)
    for category in categories:
        classPrior = arrTrainedClassPriors[category]
        dfProbPerTestInstPerClass[category]=classPrior
        # Check for each row
        for nRow in range(XTest.shape[0]):

            # Multiply the current given probability by the newly calculated probability for the given event (feature)
            for nCol in range(XTest.shape[1]):
                xFeatureVal=XTest.iloc[nRow, nCol]
                mean=dfTrainedMean.iloc[category,nCol]
                std=dfTrainedStd.iloc[category,nCol]
                gaussianProb = calcGaussianProb(xFeatureVal, mean, std)
                # multiple the prior class probability with the gausian likelihood:
                dfProbPerTestInstPerClass.iloc[nRow, category] *= gaussianProb
    return dfProbPerTestInstPerClass

In [50]:

def predictClasses(df_probPerTestInstPerClass):
    res=pd.Series(index=df_probPerTestInstPerClass.index)
    for row in df_probPerTestInstPerClass.index:
        res[row]=df_probPerTestInstPerClass.loc[row].idxmax()
    print (res)
    return res

In [51]:
def predict(XTest, arrTrainedClassPriors, dfTrainedMean, dfTrainedStd, categories):
    # 1. calculate a posterior probabities:
    dfProbPerTestInstPerClass = calcAposteriorProbs(XTest, arrTrainedClassPriors, dfTrainedMean, dfTrainedStd, categories)

    # 2. predict classes using the a posterior probabities:
    results = predictClasses(dfProbPerTestInstPerClass)
    
    return results

In [52]:

def evaluate_accuracy(y_true, y_pred):
    """
    Compare how many predictions were correct (compare the y_hat to y)
    """
    accuracy_score = pd.Series(y_true.values == y_pred.values).value_counts() * 100 / y_true.shape[0]
    return accuracy_score.iloc[0]

In [53]:
# --------------------- 
## The Following tests the the predict, using the accuracy function
# --------------------- 
meanLiklihoodDf, stdLiklihoodDf, arrPriors=fit(train_set)
iris_classes = np.unique(train_set['target'].values)
y_hat = predict(X_test, arrPriors, meanLiklihoodDf, stdLiklihoodDf, iris_classes)
accuracy_score = evaluate_accuracy(y_test['target'], y_hat)
assert accuracy_score == 100, "accuracy should be 100"
print("Accuracy Score: {}".format(accuracy_score))

73     1.0
18     0.0
118    2.0
78     1.0
76     1.0
31     0.0
64     1.0
141    2.0
68     1.0
82     1.0
110    2.0
12     0.0
36     0.0
9      0.0
19     0.0
56     1.0
104    2.0
69     1.0
55     1.0
132    2.0
29     0.0
127    2.0
26     0.0
128    2.0
131    2.0
145    2.0
108    2.0
143    2.0
45     0.0
30     0.0
dtype: float64
Accuracy Score: 100.0
